# IndexIVFPQ

更低的内存占用

我们看到的索引IndexFlatL2和IndexIVFFlat都存储完整的向量。 为了扩展到非常大的数据集，Faiss提供了基于产品量化器的有损压缩来压缩存储的向量的变体。压缩的方法基于乘积量化([Product Quantization for Nearest Neighbor Search](https://hal.inria.fr/file/index/docid/514462/filename/paper_hal.pdf))。
在这种情况下，由于矢量没有精确存储，搜索方法返回的距离也是近似值。

## 完整代码

In [2]:
import numpy as np

d = 64                              # 向量维度
nb = 100000                         # 向量集大小
nq = 10000                          # 查询次数
np.random.seed(1234)                # 随机种子,使结果可复现
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

import faiss

nlist = 100
m = 8
k = 4
quantizer = faiss.IndexFlatL2(d)    # 内部的索引方式依然不变
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
                                    # 每个向量都被编码为8个字节大小
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], k)      # 测试
print(I)
print(D)

index.nprobe = 10                   # 与以前的方法相比
D, I = index.search(xq, k)          # 检索
print(I[-5:])

[[   0   78  608  159]
 [   1 1063  555  380]
 [   2  304  134   46]
 [   3   64  773  265]
 [   4  288  827  531]]
[[1.6157436 6.1152253 6.4348025 6.564184 ]
 [1.389575  5.6771317 5.9956017 6.486294 ]
 [1.7025063 6.121688  6.189084  6.489888 ]
 [1.8057687 6.5440307 6.6684756 6.859398 ]
 [1.4920276 5.79976   6.190908  6.3791513]]
[[ 9900  8746  9853 10437]
 [10494 10507 11373  9014]
 [10719 11291 10424 10138]
 [10122  9638 11113 10630]
 [ 9229 10304  9644 10370]]


可以看到，最近的邻居被正确地找到（它是矢量ID本身），但向量自身的估计距离不是0，尽管它远远低于与其他邻居的距离。这是由于有损压缩。

当搜索真实查询时，虽然结果大多是错误的(与刚才的IVFFlat进行比较)，但是它们在正确的空间区域，而对于真实数据，情况更好，因为：
- 统一数据很难进行索引，因为没有规律性可以被利用来聚集或降低维度
- 对于自然数据，语义最近邻居往往比不相关的结果更接近。

## 工厂

由于构建索引可能会变得复杂，因此faiss提供工厂函数用于接受一个字符串来构造响应的索引。上面的索引可以通过以下简写获得：
```python
index = faiss.index_factory（d，“IVF100，PQ8”）
```

更换PQ4用Flat得到的IndexFlat。当预处理（PCA）应用于输入向量时，工厂特别有用。

例如，预处理的工厂字符串通过PCA投影将矢量减少到32维为："PCA32,IVF100,Flat"。